In [ ]:
# CREACIÓN DEL FONDO ESTÁTICO
import cv2
import numpy as np
import matplotlib.pyplot as plt

video = cv2.VideoCapture("trafico.mp4")
# Variables para acumular frames
accumulated_frame = None
frame_count = 0

while True:
    ret, frame = video.read()
    if not ret:
        break  # Fin del video

    cv2.imshow('Video', frame)  # Muestra el frame en una ventana

    # Espera 25 ms y sale si se presiona la tecla 'q'
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

    # Convierte el frame a float32 para acumulación
    frame_float = frame.astype(np.float32)

    if accumulated_frame is None:
        accumulated_frame = frame_float
    else:
        accumulated_frame += frame_float
    frame_count += 1

video.release()

# Calcula el promedio
background = accumulated_frame / frame_count

# Convierte de nuevo a uint8 para mostrar/guardar
background_uint8 = cv2.convertScaleAbs(background)

# Muestra el fondo promedio
cv2.imshow('Fondo Promedio', background_uint8)
# cv2.imwrite('fondo_estatico.jpg', background_uint8)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# UMBRALIZACIÓN Y BINARIZACIÓN DEL VIDEO
import cv2
import numpy as np
import matplotlib.pyplot as plt

fondo = cv2.imread("fondo_estatico.jpg") #Leer la imagen del fondo.
fondo_BW = cv2.cvtColor(fondo, cv2.COLOR_BGR2GRAY) #Pasar el fondo a blanco y negro.

video = cv2.VideoCapture("trafico.mp4")
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

while True:
    ret, frame = video.read()
    if not ret:
        break

    # Convertir el frame a escala de grises
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Restar el fondo al frame actual (diferencia absoluta)
    diff = cv2.absdiff(frame_gray, fondo_BW)

    # Aplicar un umbral para detectar movimiento (valores mayores a 30, por ejemplo)
    T = 30
    _, thresh = cv2.threshold(diff, T, 255, cv2.THRESH_BINARY)
    #cv2.imshow('Diferencia Absoluta', diff)
    #cv2.imshow('Movimiento Detectado', thresh)

# DETECCIÓN DE BLOBS EN EL VIDEO    

    # Limpiar ruido con operaciones morfológicas
    kernel = np.ones((5, 5), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_DILATE, kernel)
    contours, _ = cv2.findContours(cleaned, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 300:
            continue  # Ignoramos áreas pequeñas

        x, y, w, h = cv2.boundingRect(cnt)

        # Adaptar área mínima en función de la posición vertical (y)
        y_center = y + h // 2
        vertical_factor = y_center / frame_height
        min_area = 300 + (1 - vertical_factor) * 1500  # dinámico: 300 a 1800

        if area > min_area:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

#cv2.imshow('Frame con Blobs', frame)
#cv2.imshow('Máscara de Movimiento', cleaned)
# RESULTADO
    cv2.imshow('Deteccion de Coches', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()